<a href="https://colab.research.google.com/github/DataBytes-Organisation/Fine-Tuning-LLMs-for-Enterprise-Applications/blob/ed_branch/Ed_LLM_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Personalized Healthcare QA System (Chatbot)
#### Ed:215279167
---
### Objective:
Develop a healthcare chatbot that answers patient inquiries on medications, symptoms, and  treatments while reducing hallucinations.

### Datasets:
* MedicineQuAD – Medication-focused QA dataset.
* MedQA (USMLE) – Medical board exam Q&A.
* PubMedQA – Biomedical QA dataset.

### Task Breakdown:
1. Train models on medical QA datasets.
2. Fine-tune for patient-friendly responses (simplified, clear language).
3. Ensure context-aware, regulatory-compliant answers:  
o Implement FDA/TGA guideline alignment.
o Develop a retrieval-based validation for generated answers.
4. Deploy as a chatbot interface (React-based UI + API integration).
5. Implement real-time fact-checking:  
o Confidence score visualization (green = high confidence, yellow = medium, red = low
confidence).
o Integration with PubMed and trusted medical sources.

### Models to Use:
• Llama-2 7B, Llama-2 13B, Falcon 7B, Mistral 7B, Mistral 8×7B, GPT-3.5-Turbo, Gemini Pro 1.0,
Gemini 1.5 Pro, Gemini 1.5 Flash, Gemini 1.5 Flash 8B, Gemini 2.0 Flash Experimental, Flan
T5 Large, Flan-T5 XL

### Evaluation Metrics:
* Medical factual accuracy (expert validation).
* Answer clarity and patient-friendliness (user study assessment).
* Trustworthiness Score (alignment with FDA/TGA guidelines).

---
### 1. Import libraries and model to prepaare for fine tuning

In [1]:
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer
import transformers
import torch

In [2]:
# using hugging face to download model
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [3]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [4]:
# initialise tokenizer and pipeline
tokenizer = AutoTokenizer.from_pretrained(model, token=hf_token)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [6]:
# prompt with test query to validate llm working
sequences = pipeline(
  """
  How can I get rid of the flu?
  """,
  do_sample=True,
  top_k=10,
  num_return_sequences=1,
  eos_token_id=tokenizer.eos_token_id,
  truncation=True,
  max_length=300
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: 
  How can I get rid of the flu?
  
  There are several ways to help get rid of the flu, including:
  
  Over-the-counter medications: These medications can help relieve symptoms such as fever, headache, and body aches. They work by reducing the inflammation in the body and can help you feel better faster.
  
  Home remedies: There are several home remedies that can help get rid of the flu, such as drinking plenty of fluids, getting enough rest, and using a humidifier to help loosen congestion.
  
  Antiviral medications: These medications can help reduce the severity and duration of the flu by targeting the virus itself. They are most effective when started within the first 48 hours of symptoms.
  
  Rest: Getting plenty of rest can help your body fight off the flu virus.
  
  Staying hydrated: Drinking plenty of fluids can help thin out mucus and make it easier to breathe.
  
  Overall, the best way to get rid of the flu is to take a combination of these approaches.




### Implementation Strategy
1. **Fine-Tuning Approach**
* Use Supervised Fine-Tuning (SFT) and LoRA for efficient training.
* Train each model separately, evaluating before and after fine-tuning.
2. **Dataset Preprocessing**
* Tokenization & data augmentation for question-answer pairs.
* Alignment filtering to remove conflicting data.
3. **Model Training Pipeline**
* Step 1: Train models with pre-existing medical QA datasets.
* Step 2: Implement LoRA-based fine-tuning to inject domain-specific expertise.
* Step 3: Evaluate hallucination/misinformation rates pre- and post-fine-tuning.
* Step 4: Deploy chatbot interface for real-time healthcare QA.
4. **Chatbot Deployment for Personalized Healthcare QA**
* Frontend: React-based UI.
* Backend: Flask/FastAPI + model hosting on cloud GPUs.
* Integration: Fact-checking module with real-time verification.
* Confidence Display: Answers are color-coded based on accuracy scores.

### Expected Outcomes
AI-Powered Healthcare Chatbot:
* Patient-friendly responses with regulatory compliance.
* Real-time fact-checking for accurate health guidance.

### Data Selection and Training Strategy
Datasets Suggested:
* MedicineQuAD – Medication-focused QA dataset.
* MedQA (USMLE) – Medical board exam Q&A dataset.
* PubMedQA – Biomedical QA dataset.
* MedDialog – Doctor-patient conversation dataset.

Training Strategy:
Use different datasets for different chatbot functions:
* For general medical QA → Use MedQA + PubMedQA.
* For medication-specific queries → Use MedicineQuAD.
* For conversational style training → Use MedDialog.

Why?
* MedDialog fine-tunes the chatbot’s conversational style.
* MedQA and PubMedQA ensure medical factual accuracy.
* MedicineQuAD specializes in pharmaceutical knowledge.
➡ Recommendation:

Use all datasets, but weight them differently:
* Core Training: MedQA + MedicineQuAD.
* Evaluation: PubMedQA + MedDialog.